In [0]:
import pandas as pd
import numpy as np

Get the main synced files, drop irrelevant columns and discard entries that don't have values for attention files

Need schoolg.txt (syncd file), user_records folder, and schoolg directory with all the brainwave data. Same for the schooln data.

In [0]:
schoolg = pd.read_csv("https://attention-difficulty.s3.us-east-2.amazonaws.com/synced/schoolg.txt",delimiter=',')
print("schoolg shape", schoolg.shape)
schooln = pd.read_csv('https://attention-difficulty.s3.us-east-2.amazonaws.com/synced/schooln.txt',delimiter=',')
print("schooln shape", schooln.shape)

schoolg shape (4468, 10)
schooln shape (24125, 10)


In [0]:
print(f"schoolg: Dropping {schoolg.shape[0] - schoolg.dropna(subset=['brainwave_file_path_attention']).shape[0]} rows without corresponding brainwave data")

schoolg = schoolg.dropna(subset=['brainwave_file_path_attention'])
schoolg = schoolg.drop(columns=['brainwave_file_path_EEG', 'brainwave_file_path_events'])
print(schoolg.shape)

print(f"schooln: Dropping {schooln.shape[0] - schooln.dropna(subset=['brainwave_file_path_attention']).shape[0]} rows without corresponding brainwave data")

schooln = schooln.dropna(subset=['brainwave_file_path_attention'])
schooln = schooln.drop(columns=['brainwave_file_path_EEG', 'brainwave_file_path_events'])
print(schooln.shape)

# schooln = schooln.dropna(subset=['brainwave_file_path_attention'])
# schooln = schooln.drop(columns=['brainwave_file_path_EEG', 'brainwave_file_path_events'])

# stacked = pd.concat([schoolg, schooln])

schoolg: Dropping 4088 rows without corresponding brainwave data
(380, 8)
schooln: Dropping 21093 rows without corresponding brainwave data
(3032, 8)


We still need to get the correctness and difficulty for each question, as well as the attention. However, matching timestamped attention may be difficult due to data inconsistencies. The next steps go through with joining each subject's question difficulty with each student's response information from the schoolg and schooln data. Then we calculate average attention readings using the attention data files and add an avg_attention column to reflect these values. This is coarse information that can be used to loosely connect difficulty and attention.

In [0]:
# change for the real thing
df = schoolg 
print(len(schoolg))
display(df.dtypes)
display(df[df['subject']=='en'].head())

380


user_id                           int64
subject                          object
question_id                      object
school_id                         int64
is_right                          int64
stime                            object
ctime                            object
brainwave_file_path_attention    object
dtype: object

,user_id,subject,question_id,school_id,is_right,stime,ctime,brainwave_file_path_attention
1237,6840100809585686,en,192136,196,1,29-Dec-2018 08:39:31,29-Dec-2018 08:40:12,schoolg/20181229-08-30/attention_f0fe6b76a0ec.log
1238,6840100809585686,en,192151,196,1,29-Dec-2018 08:40:13,29-Dec-2018 08:40:32,schoolg/20181229-08-30/attention_f0fe6b76a0ec.log
1239,6840100809585686,en,192150,196,1,29-Dec-2018 08:40:33,29-Dec-2018 08:41:00,schoolg/20181229-08-30/attention_f0fe6b76a0ec.log
1240,6840100809585686,en,85034,196,1,29-Dec-2018 08:41:01,29-Dec-2018 08:41:39,schoolg/20181229-08-30/attention_f0fe6b76a0ec.log
1241,6840100811464495,en,275568,196,0,29-Dec-2018 08:39:25,29-Dec-2018 08:41:47,schoolg/20181229-08-30/attention_f0fe6b55e328.log


In [0]:
df = pd.concat([schoolg, schooln]) 

In [0]:
# get the question information by referencing the subject user records

def get_subject_data(url):
  df = pd.read_csv(url)
  df = df[['user_id', 'school_index','question_id', 'difficulty', 'is_right']]
  df.question_id = df.question_id.astype(str)
  return df

math = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/math_record_cleaned.csv')

chemistry = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/chem_record_cleaned.csv')

cn = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/cn_record_cleaned.csv')

phy = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/phy_record_cleaned.csv')

reading = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/en_reading_record_cleaned.csv')

en = get_subject_data('https://attention-difficulty.s3.us-east-2.amazonaws.com/user+records/en_record_cleaned.csv')

subjects = pd.concat([math, chemistry, cn, phy, reading, en]).drop_duplicates(subset=['user_id', 'question_id'])
subjects.head()
subjects.dtypes

user_id          int64
school_index     int64
question_id     object
difficulty       int64
is_right         int64
dtype: object

In [0]:
# join with original data by matching user_id and question_id, now we have difficulty and correctness info
new_df = pd.merge(df, subjects, how='left', on=['user_id','question_id'])

In [0]:
def attentionfile_to_s3(x):
  idx = x.find('school') + len('schoolX/')
  return 'https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/' + x[:idx] + 'data/' + x[idx:]

new_df['s3_brainwave_file_path_attention'] = new_df['brainwave_file_path_attention'].apply(attentionfile_to_s3)

In [0]:
# def calculate_avg_attention(filename):
#   try:
#     print(f'reading {filename}\r')
#     log = pd.read_csv(filename, delimiter='\t', skiprows=2, names=['time', 'attention'])
#     print(f'done   \r')
#     return log.attention.mean()
#   except OSError as e:
#     # print('file not found: ', filename)
#     return -1

In [0]:
# new_df['avg_attention'] = new_df['s3_brainwave_file_path_attention'].apply(calculate_avg_attention)
# display(f"could not find {new_df[new_df['avg_attention'] == -1].shape[0]} out of {new_df.shape[0]} attention files")
# attention = new_df[new_df['avg_attention'] != -1]
# display(attention.shape)

In [0]:
new_df.head()

,user_id,subject,question_id,school_id,is_right_x,stime,ctime,brainwave_file_path_attention,school_index,difficulty,is_right_y,s3_brainwave_file_path_attention
0,6840100808795007,math,590bdec1f4aeb5737e1de73b,196,0,02-Dec-2018 14:50:24,02-Dec-2018 14:54:10,schoolg/20181202-14-39/attention_f0fe6b76a13a.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...
1,6840100808795007,math,590bdec1f4aeb5737e1de73a,196,0,02-Dec-2018 14:54:32,02-Dec-2018 14:58:26,schoolg/20181202-14-39/attention_f0fe6b76a13a.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...
2,6840100808795007,math,590bdec1f4aeb5737e1de74a,196,0,02-Dec-2018 14:58:27,02-Dec-2018 15:00:29,schoolg/20181202-14-39/attention_f0fe6b76a13a.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...
3,6840100808795007,math,590bdec1f4aeb5737e1de749,196,1,02-Dec-2018 15:00:29,02-Dec-2018 15:01:26,schoolg/20181202-14-39/attention_f0fe6b76a13a.log,1,2,1,https://attention-difficulty.s3.us-east-2.amaz...
4,6840100808795007,math,590bdec1f4aeb5737e1de74b,196,0,02-Dec-2018 15:01:27,02-Dec-2018 15:03:28,schoolg/20181202-14-39/attention_f0fe6b76a13a.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...


In [0]:
new_df.sort_values(['user_id', 'stime']).head()

,user_id,subject,question_id,school_id,is_right_x,stime,ctime,brainwave_file_path_attention,school_index,difficulty,is_right_y,s3_brainwave_file_path_attention
3199,44749,cn,31648,290,1,01-Dec-2018 08:13:25,01-Dec-2018 08:17:58,schooln/20181201-15-59/attention_f0fe6b55e1d2.log,1,2,1,https://attention-difficulty.s3.us-east-2.amaz...
3201,44749,cn,31649,290,0,01-Dec-2018 08:17:58,01-Dec-2018 08:22:18,schooln/20181201-15-59/attention_f0fe6b55e1d2.log,1,3,0,https://attention-difficulty.s3.us-east-2.amaz...
3202,44749,cn,31647,290,0,01-Dec-2018 08:22:19,01-Dec-2018 08:25:03,schooln/20181201-15-59/attention_f0fe6b55e1d2.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...
3206,44749,cn,31653,290,0,01-Dec-2018 08:46:26,01-Dec-2018 08:59:09,schooln/20181201-15-59/attention_f0fe6b55e1d2.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...
3208,44749,cn,31654,290,0,01-Dec-2018 08:59:15,01-Dec-2018 09:03:07,schooln/20181201-15-59/attention_f0fe6b55e1d2.log,1,2,0,https://attention-difficulty.s3.us-east-2.amaz...


In [0]:
new_df.to_csv('mutla-all-subj-avg-attention.csv')

In [0]:
from google.colab import files 
files.download('mutla-all-subj-avg-attention.csv')

In [0]:
# calculate average attention between stime and ctime

def calculate_specific_attention(filename, stime, ctime):
    try:
        log = pd.read_csv(filename, delimiter='\t', skiprows=2, names=['time', 'attention'])
        log['time'] = pd.to_datetime(log['time'], unit='s') + pd.Timedelta(hours=8)
        mask = (log['time'] > stime) & (log['time'] <= ctime)
        question_attention_score = log[mask]['attention'].mean()
        return question_attention_score
    
    except OSError as e:
        print('file not found: ', filename)
    except ValueError as e:
        print('unable to get datetime:', filename)

In [0]:
test_df = new_df.iloc[100:105, :]


In [0]:
new_df['specific_attention'] = new_df.apply(lambda x: calculate_specific_attention(x['s3_brainwave_file_path_attention'], x['stime'], x['ctime']), axis=1)

unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0fe6b55e21c.log
unable to get datetime: https://attention-difficulty.s3.us-east-2.amazonaws.com/brainwave/schooln/data/20181211-17-34/attention_f0

In [0]:
new_df.to_csv('mutla-final-data.csv')

In [0]:
from google.colab import files 
files.download('mutla-final-data.csv')